<a href="https://colab.research.google.com/github/halen48/HarmonyShifter/blob/main/Harmony_Shifter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Harmony Shifter in Python</h1>
Author: <a href="https://github.com/halen48">Guilherme Novaes</a>

---
Refereces: 
*   <a href="https://hellomusictheory.com/learn/negative-harmony/">What Is Negative Harmony?</a>
*   <a href="https://en.wikipedia.org/wiki/Riemannian_theory">Riemannian theory</a>


In [333]:
from google.colab import files
!pip install mido
from mido import MidiFile
import os

In [334]:
ZIP_NAME = "shifted_harmonies.zip" #@param
#Clean the work dir
for f in os.listdir():
  if(os.path.isdir(f)):
    continue
  os.remove(f)

#1) Implementation

##Constants/Formulas

In [335]:
SHARP = '♯'
FLAT = '♭'

<a href="https://www.inspiredacoustics.com/en/MIDI_note_numbers_and_center_frequencies">MIDI Notes</a>

In [336]:
def mid2note(v):
  return ((v%12) ,v//12 - 1)

Channel 10 is always percurssion.<br>
In other words, the 10th channel is the percussion.<br>
Since first channel by mido is the 0 channel, the 10th channel is the number 9

In [337]:
PERCUSSION_CHANNEL = 10

In [338]:
def isPercussion(tracks):
  for m in tracks:
    dict_ = m.dict()
    if('channel' in dict_.keys()):
      return (dict_['channel'] == PERCUSSION_CHANNEL - 1)
  
  return False

Statistics to get the harmonic field of the music

In [339]:
def track_notes_statistics(track):
  ret = {}
  if(isPercussion(track)):
    return ret
  for p in track:
    try:
      note = mid2note(p.note)[0]
      if(not note in ret):
        ret[note] = p.time
      else:
        ret[note] += p.time
    except AttributeError:
      #Ignore the objects that isn't note messages in the midi
      pass
  
  if(not len(ret)):
    return []
  base = max([(ret[key]) for key in ret])
  #Sorting descending frequency (most first)
  return sorted([(key, ret[key]/base) for key in ret], key=lambda a : -a[1])

##Chromatic Scale

In [340]:
class ChromaticScale():
  def __init__(self, default_accidental):
    
    self.__ACCIDENTAL = ['sharp', 'flat']
    self.__default_accidental = default_accidental
    self.__default_accidental_index = self.__ACCIDENTAL.index(default_accidental)
    
    self._notes = {'sharp' : [], 'flat' : [], 'note2idx' : {}, 'idx2note' : {}}
    for i in range(ord('A'),ord('G')+1):
      self._notes['sharp'].append(chr(i))
      if(i != ord('B') and i!= ord('E')):
        self._notes['sharp'].append(chr(i)+SHARP)
    
    self._MAX_NOTES = len(self._notes['sharp'])

    self._notes['flat'] = [chr( ord('A') + (ord(v[0])- ord('A')+1)%7)+FLAT if len(v) > 1 else v for v in self._notes['sharp']]
    
    self.rotate('C',False)
  
    for i,note in enumerate(self._notes[default_accidental]):
      self._notes['note2idx'][note] = i
      self._notes['idx2note'][i] = note

    self.__circle5ths()

  def __circle5ths(self):
    self._circle = {'sharp' : [self._notes['sharp'][0]], 'flat' : [self._notes['flat'][0]], 'discrete' : [0], 'map' : {} }
    index = 7
    while index != 0:
      self._circle['sharp'].append(self._notes['sharp'][index])
      self._circle['flat'].append(self._notes['flat'][index])
      self._circle['discrete'].append(index)
      index = (index+7)%self._MAX_NOTES
    
    middle_slice = j = self._MAX_NOTES//2
    
    for i in range(1,middle_slice+1):
      note1, note2 = self.notes()[self._circle['discrete'][i]], self.notes()[self._circle['discrete'][j-middle_slice]]
      note1 = self._notes['note2idx'][note1] 
      note2 = self._notes['note2idx'][note2]
      
      self._circle['map'][note1] = note2
      self._circle['map'][note2] = note1
      j -= 1

  def rotate(self,start_note, do_circle = True):
    start_note = start_note.replace('#','♯').replace('b','♭')
    if(start_note in self._notes['sharp']):
      notes = self._notes['sharp']
    else:
      notes = self._notes['flat']
    while notes[0] != start_note:
      self._notes['sharp'].append(self._notes['sharp'].pop(0))
      self._notes['flat'].append(self._notes['flat'].pop(0))
    
    if(do_circle):
      self.__circle5ths()

  def print_circle(self, pettry = False):
    major = self._circle[self.__default_accidental].copy()
    self.rotate('A')
    minor = self._circle[self.__default_accidental].copy()
    self.rotate('C')
    ret = {'major':major, 'minor' : minor}
    if(pettry):
      return '\n'.join(['%s: %s'%(key,ret[key]) for key in ret])
    return ret
  
  def notes(self):
    return self._notes[self.__default_accidental]    
  
  def circle(self,key):
    return self._circle[key]

  def negativeMap(self):
    return '\n'.join(["%2s → %2s"%(self._notes['idx2note'][key],self._notes['idx2note'][self._circle['map'][key]]) for key in self._circle['discrete']])

  def accidental(self):
    return self.__default_accidental
    

##N-tonic Scale
Intervals: 
*  [W]hole = 2
*  [H]alf-step = 1

In [341]:
class NTonicScale(ChromaticScale):
    def __init__(self, intervals, default_accidental = 'sharp'):
      super().__init__(default_accidental)
      intervals = [int(v) for v in intervals.replace('–','-').split('-')]
      self.indexes = [0]
      for interval in intervals:
        if(self.indexes[-1]+interval >= len(self.notes())):
          break
        self.indexes.append(self.indexes[-1]+interval)
      self.scale = {'notes':None, 'discrete' : None}
      self.scale_notes('C')
      
    def scale_notes(self, root):
      self.rotate(root)
      self.scale['notes'] = [self.notes()[v] for v in self.indexes]
      self.scale['discrete'] = []
      self.scale['discrete'] = [self._notes['note2idx'][v] for v in self.scale['notes']]

      return self
    
    def fit(self, notes_freqs : list, root : int) -> bool:
      self.scale_notes(self._notes['idx2note'][root])
      notes = [note[0] for note in notes_freqs]
      #Check if all indexes are inside of list [notes]
      probability = 0
      for index in self.scale['discrete']:
        try:
          idx = notes.index(index)
          probability += notes_freqs[idx][1]
        except ValueError:
          #Just ignore
          pass
      
      probability /= len(self.scale['discrete'])

      return probability

    def __str__(self):
      return ' '.join([v for v in self.scale['notes']])
        

##Circle of fifiths


In [342]:
#@title <img src="https://lh5.googleusercontent.com/jq0aEjROUrEuCw6aJS84iCzEd5jm707BRXFckMJ22VXm30HOHcP6vyKMEqPH4dVADi7FvOUu7a23C9PbH4aafuSbrdYRwlVCEAhscIGwbzxtSNFU5TDSairq_NlOvcH_H3bQSbXF" width="300"/>
chromatic = ChromaticScale('sharp')
print(chromatic.notes())
print(chromatic.print_circle(True))
print(chromatic.negativeMap())

['C', 'C♯', 'D', 'D♯', 'E', 'F', 'F♯', 'G', 'G♯', 'A', 'A♯', 'B']
major: ['C', 'G', 'D', 'A', 'E', 'B', 'F♯', 'C♯', 'G♯', 'D♯', 'A♯', 'F']
minor: ['A', 'E', 'B', 'F♯', 'C♯', 'G♯', 'D♯', 'A♯', 'F', 'C', 'G', 'D']
 C →  G
 G →  C
 D →  F
 A → A♯
 E → D♯
 B → G♯
F♯ → C♯
C♯ → F♯
G♯ →  B
D♯ →  E
A♯ →  A
 F →  D


##Scales

In [343]:
scales = {
    'Diatonic': {
      'Ionian': '2–2–1–2–2–2–1',
      'Dorian': '2–1–2–2–2–1–2',
      'Phrygian': '1–2–2–2–1–2–2',
      'Lydian': '2–2–2–1–2–2–1',
      'Mixolydian': '2–2–1–2–2–1–2',
      'Aeolian': '2–1–2–2–1–2–2',
      'Locrian': '1–2–2–1–2–2–2',
    },
    'Triads': {
      'Major Triad' : '4-3-5',
      'Minor Triad' : '3-4-5',
    }
}

In [344]:
def scales_fit(notes : list, group : str, key : int) -> list:
  return sorted([(name,scales[group][name].fit(notes, key)) for name in scales[group]],key= lambda a : -a[1])

In [345]:
#@title <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/31/Modal_Interval_Sequence.png/1024px-Modal_Interval_Sequence.png" width="600">
for group in scales:
  print("====== %s ======"%group)
  for name in scales[group]:
    scales[group][name] = NTonicScale(scales[group][name], chromatic.accidental())
    print("%s: %s"%(name, scales[group][name]))

====== Diatonic ======
Ionian: C D E F G A B
Dorian: C D D♯ F G A A♯
Phrygian: C C♯ D♯ F G G♯ A♯
Lydian: C D E F♯ G A B
Mixolydian: C D E F G A A♯
Aeolian: C D D♯ F G G♯ A♯
Locrian: C C♯ D♯ F F♯ G♯ A♯
====== Triads ======
Major Triad: C E G
Minor Triad: C D♯ G


##Implement your shift-harmonies formulas here

###All methods with the sufix below will be automatically included to perform the operation
```
music_info = {'tracks' : [], 'root' : None, 'scale' : None}
```

In [346]:
SUFIX = 'Harmony'

In [347]:
class Harmonies(): 
  operations = {}
  dynamic_functions = {}
  def fifithHarmony(note : int, music_info : dict, key : str) -> int:
    v = mid2note(note)
    return chromatic.circle('discrete')[v[0]-1] + (12*(1+v[1]))

  def negativeHarmony(note, music_info, key):
    v = mid2note(note)
    return chromatic.circle('map')[v[0]] + (12*(1+v[1]))

####Scale shift functions

In [348]:
def scale_shift(note, music_info, group, target_scale):
  target_scale = target_scale.replace("_%s",'')
  if(music_info['scale'] == target_scale):
    return note
  
  v = mid2note(note)

  #idenfity the note index on the scale
  root, old_scale = scales[group][music_info['scale']].scale['notes'][0], scales[group][music_info['scale']]
  new_scale = scales[group][target_scale]

  old_notes = old_scale.scale['discrete']
  new_notes = new_scale.scale_notes(root).scale['discrete']
  try:
    idx = old_notes.index(v[0])
  except ValueError:
    #ignore and return the same note
    return note
  
  #idenfity the index of the note of the shifted scale
  new_note = new_notes[idx]
  #print(chromatic.notes()[v[0]], music_info['scale'], "->", target_scale,chromatic.notes()[new_note])

  return new_note + (12*(1+v[1]))

In [349]:
for scale in scales['Diatonic']:
  Harmonies.dynamic_functions[scale+SUFIX] = lambda a,b, key : scale_shift(a,b, 'Diatonic', key)

In [350]:
for key in Harmonies.__dict__:
  if(SUFIX in key):
    Harmonies.operations[key.replace(SUFIX,'_%s')] = Harmonies.__dict__[key]

In [351]:
for key in Harmonies.dynamic_functions:
    if(SUFIX in key):
      Harmonies.operations[key.replace(SUFIX,'_%s')] = Harmonies.dynamic_functions[key]

#2) Upload a MIDI

In [352]:
#@title Optional (You can upload multiple)
uploaded = files.upload() 

Saving Van_Halen_-_Jump.mid to Van_Halen_-_Jump.mid


##Do the mathmagics

In [353]:
FOLDER = "generated/"

In [354]:
try:
  os.mkdir(FOLDER)
except FileExistsError:
  pass

In [355]:
dir_files = os.listdir()

In [356]:
music_info = {}

###Get the valid tracks with notes

In [357]:
mid_files = []
for file_name in dir_files:
  if(file_name[-4:] != '.mid'):
    continue
  mid_files.append(file_name)
  music_info[file_name] = {'tracks' : [], 'root' : None, 'scale' : None}
  mid = MidiFile(file_name, clip=True)
  for track in mid.tracks:
    if(not isPercussion(track)):
      music_info[file_name]['tracks'].append(track)

###Check the main scale of the song

####Get the root

In [358]:
for file_name in mid_files:
  best_root =  {}
  for i,v in enumerate(music_info[file_name]['tracks']):
    notes = track_notes_statistics(v)
    if(not notes):
      continue
    most_played_note = notes[0][0]
    
    if(not most_played_note in best_root):
      best_root[most_played_note] = notes[0][1]
    else:
      best_root[most_played_note] += notes[0][1]
  
  music_info[file_name]['root'] = sorted([(key, best_root[key]/len(music_info[file_name]['tracks'])) for key in best_root], key=lambda a : -a[1])[0][0]

####Get the scale

In [359]:
for file_name in mid_files:
  song_scale = {}
  for i,v in enumerate(music_info[file_name]['tracks']):
    notes = track_notes_statistics(v)
    if(not notes):
      continue

    ret = scales_fit(notes, 'Diatonic', music_info[file_name]['root'])
    
    for scale in ret[:3]:
      if(not scale[0] in song_scale):
        song_scale[scale[0]] = scale[1]
      else:
        song_scale[scale[0]] += scale[1]
  
  song_scale = sorted([(key, song_scale[key]/len(music_info[file_name]['tracks'])) for key in song_scale], key=lambda a : -a[1])
  
  music_info[file_name]['scale'] = song_scale[0][0]
  print("Scales of Song: %s"%(file_name))
  for scale in song_scale[:3]:
    print("\t\t> %s %s (%.2f%%)"%(chromatic.notes()[music_info[file_name]['root']],scale[0], 100*scale[1]))

Scales of Song: Van_Halen_-_Jump.mid
		> C Mixolydian (17.48%)
		> C Ionian (15.14%)
		> C Dorian (8.58%)


###Process the harmonies

In [360]:
file_list_download = []
for name in Harmonies.operations:
  for file_name in mid_files:
    if(file_name[-4:] != '.mid'):
      continue

    mid = MidiFile(file_name, clip=True)

    for track in mid.tracks:
      if(isPercussion(track)):
        continue
      for value in track:
        try:
          value.note = Harmonies.operations[name](value.note, music_info[file_name],name)
        except AttributeError:
          #Ignore the objects that isn't note messages in the midi
          pass
    
    file_list_download.append(FOLDER+(name%file_name))
    mid.save(file_list_download[-1])

#3) Download

In [361]:
!zip -rm $ZIP_NAME $FOLDER
files.download(ZIP_NAME)

  adding: generated/ (stored 0%)
  adding: generated/Ionian_Van_Halen_-_Jump.mid (deflated 58%)
  adding: generated/fifith_Van_Halen_-_Jump.mid (deflated 58%)
  adding: generated/Lydian_Van_Halen_-_Jump.mid (deflated 58%)
  adding: generated/negative_Van_Halen_-_Jump.mid (deflated 58%)
  adding: generated/Aeolian_Van_Halen_-_Jump.mid (deflated 58%)
  adding: generated/Locrian_Van_Halen_-_Jump.mid (deflated 58%)
  adding: generated/Mixolydian_Van_Halen_-_Jump.mid (deflated 58%)
  adding: generated/Phrygian_Van_Halen_-_Jump.mid (deflated 58%)
  adding: generated/Dorian_Van_Halen_-_Jump.mid (deflated 58%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>